In [1]:
import os
os.path.expanduser('~/.keras/models')
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
model_conv = ResNet152(weights='imagenet', input_shape=(150, 150, 3), include_top=False)

In [3]:
model_conv.summary()

Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
__________________________________________________________________________________________

Total params: 58,370,944
Trainable params: 58,219,520
Non-trainable params: 151,424
__________________________________________________________________________________________________


In [4]:
data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
)
features = np.zeros(shape=(2000, 5, 5, 2048))
labels = np.zeros(shape=(2000))
generator = data_generator.flow_from_directory(
    'PetImages',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)
i = 0
for inputs_batch, labels_batch in generator:
    features[i*20: (i+1)*20] = model_conv.predict(inputs_batch)
    labels[i*20: (i+1)*20] = labels_batch
    i=i+1
    if i*20 >= 2000:
        break

Found 2000 images belonging to 2 classes.


In [5]:
features.shape

(2000, 5, 5, 2048)

In [6]:
labels.shape

(2000,)

In [7]:
data_generator_test = ImageDataGenerator(
    rescale=1.0/255.0,
)
features_test = np.zeros(shape=(1400, 5, 5, 2048))
labels_test = np.zeros(shape=(1400))
generator_test = data_generator_test.flow_from_directory(
    'PetImagesTest/',
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)
i = 0
for inputs_batch, labels_batch in generator_test:
    features_test[i*10: (i+1)*10] = model_conv.predict(inputs_batch)
    labels_test[i*10: (i+1)*10] = labels_batch
    i=i+1
    if i*10 >= 1400:
        break

Found 1400 images belonging to 2 classes.


In [8]:
import gc
del model_conv
gc.collect()

21723

In [9]:
features = np.reshape(features, (2000, 5 * 5 * 2048))
features_test = np.reshape(features_test, (1400, 5 * 5 * 2048))
features.shape

(2000, 51200)

In [10]:
5 * 5 * 2048

51200

In [11]:
from tensorflow.keras import models, layers, optimizers

In [12]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=51200))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
import pickle
with open('features.h5', 'wb') as f:
    pickle.dump(features, f)

In [14]:
with open('labels.h5', 'wb') as f:
    pickle.dump(labels, f)

In [15]:
with open('features_test.h5', 'wb') as f:
    pickle.dump(features_test, f)
with open('labels_test.h5', 'wb') as f:
    pickle.dump(labels_test, f)